In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder

jtplot.style(figsize=(15, 9))

In [5]:
map_data = pd.read_csv('../data/SHR65_22.csv', index_col=0)
map_data.head()

,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,Month,Incident,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
ID,,,,,,,,,,,,,,,,,,,,,
197601001AKASP00,"Juneau, AK",AKASP00,Alaska,State Troopers,Primary state LE,FBI,Yes,1976,January,1,...,American Indian or Alaskan Native,Unknown or not reported,Knife or cutting instrument,Husband,Other arguments,NaN,0,0,30180.0,Rural Alaska
197601001AL00102,"Jefferson, AL",AL00102,Alabama,Birmingham,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Acquaintance,Felon killed by private citizen,Felon killed in commission of a crime,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00104,"Jefferson, AL",AL00104,Alabama,Fairfield,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Shotgun,Wife,Other,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00106,"Jefferson, AL",AL00106,Alabama,Leeds,Municipal police,FBI,Yes,1976,January,1,...,Black,Unknown or not reported,Knife or cutting instrument,Brother,Other arguments,NaN,0,0,30180.0,"Birmingham-Hoover, AL"
197601001AL00201,"Mobile, AL",AL00201,Alabama,Mobile,Municipal police,FBI,Yes,1976,January,1,...,White,Unknown or not reported,Strangulation - hanging,Acquaintance,Circumstances undetermined,NaN,0,0,30180.0,"Mobile, AL"


In [6]:
useless_clmns = ['FileDate', 'Source', 'ActionType', 'Ori', 'Subcircum'] # 'Solved'
map_data.drop(useless_clmns, axis=1, inplace=True)

In [21]:
trans_clmns = map_data.dtypes[map_data.dtypes != 'int64'].index.to_list()

In [10]:
yes = map_data[(map_data['Solved']=='Yes')
               & ~(map_data['OffSex']=='Unknown')
               & ~(map_data['OffRace']=='Unknown')
              ].index
no = map_data[map_data['Solved']=='No'].index # test

In [4]:
yes.shape, no.shape

((602887, 23), (256868, 23))

In [5]:
# Relationship not determined / Circumstances undetermined

In [23]:
enc = OrdinalEncoder()
enc.fit(map_data[trans_clmns])
map_data[trans_clmns] = enc.transform(map_data[trans_clmns])

In [27]:
yes = map_data.loc[yes]
no = map_data.loc[no]

In [28]:
X = yes.drop('OffSex', axis=1)
y = yes['OffSex']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
offsex_model = RandomForestClassifier(max_depth=4).fit(X_train, y_train)
offsex_model.score(X_test, y_test)